## Document Ingestion and Preparation

<img style="float: right" width="800px" src="https://raw.githubusercontent.com/databricks-industry-solutions/hls-llm-doc-qa/basic-qa-LLM-HLS/images/data-prep.jpeg?token=GHSAT0AAAAAACBNXSB4IK2XJS37QU6HCJCEZEBL3TA">


#
1. Organize your documents into a Unity Catalog Volume
    * In this demo we have preuploaded a set of PDFs from PubMed on S3, but your own documents will work the same way
2. Use LangChain to ingest those documents and split them into manageable chunks using a text splitter
3. Use a sentence transformer NLP model to create embeddings of those text chunks and store them in a vectorstore
    * Embeddings are basically creating a high-dimension vector encoding the semantic meaning of a chunk of text


Start with required Python libraries for data preparation.

In [0]:
import os, sys, json

In [0]:
%run ./util/install-prep-libraries

  Using cached transformers-4.57.0-py3-none-any.whl.metadata (41 kB)
  Using cached huggingface_hub-0.35.3-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached hf_xet-1.1.10-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.7 kB)
Using cached transformers-4.57.0-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.35.3-py3-none-any.whl (564 kB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
Using cached hf_xet-1.1.10-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.30.2
    Not uninstalling huggingface-hub at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-88c91d3a-838e-430e-aea5-d5232d32cf7a
    Can't uninstall 'huggingface-hub'. No files w

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
# upgrade mlflow
%pip install "mlflow[databricks]>=3.1.0" "protobuf>=3.12.0,<5.0.0" "databricks-feature-engineering>=0.9.0" "databricks-vectorsearch>=0.50" --upgrade
dbutils.library.restartPython()

  Using cached databricks_feature_engineering-0.13.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached databricks_vectorsearch-0.60-py3-none-any.whl.metadata (2.8 kB)
  Using cached mlflow_skinny-3.4.0-py3-none-any.whl.metadata (31 kB)
INFO: pip is looking at multiple versions of databricks-feature-engineering to determine which version is compatible with other requirements. This could take a while.
  Using cached databricks_feature_engineering-0.12.1-py3-none-any.whl.metadata (4.3 kB)
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
  Using cached opentelemetry_proto-1.36.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_proto-1.35.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_proto-1.34.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_proto-1.34.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_proto-1.33.1-py3

In [0]:
import mlflow
mlflow.__version__

'3.4.0'

Creating a dropdown widget for model selection, as well as defining the file paths where our PDFs are stored, where we want to cache the HuggingFace model downloads, and where we want to persist our vectorstore.

In [0]:
# where you want the PDFs to be saved in your environment
#dbutils.widgets.text("UC_Volume_Path", "hls_llm_qa_demo.data.pdf_docs")
dbutils.widgets.text("UC_Volume_Path", "/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume")

# which embeddings model we want to use. We are going to use the foundation model API, but you can use custom models (i.e. from HuggingFace), External Models (Azure OpenAI), etc.
dbutils.widgets.text("Embeddings_Model", "databricks-bge-large-en")

# publicly accessible bucket with PDFs for this demo
#dbutils.widgets.text("Source_Documents", "s3a://db-gtm-industry-solutions/data/hls/llm_qa/")
dbutils.widgets.text("Source_Documents", "/Workspace/Users/yhu01@blueshieldca.com/adb_rag_resume/resumes_9.22")

# Location for the split documents to be saved  
#dbutils.widgets.text("Persisted_UC_Table_Location", "hls_llm_qa_demo.vse.hls_llm_qa_raw_docs")
dbutils.widgets.text("Persisted_UC_Table_Location", "dbc_adv_anlaytics_dev.yh_agent_resume.resumes_chunked_p4")

# Vector Search Endpoint Name - , hls_llm_qa_demo_vse
#dbutils.widgets.text("Vector_Search_Endpoint", "VS_ENDPOINT")
dbutils.widgets.text("Vector_Search_Endpoint", "yhu01_resume_search")

# Vector Index Name 
#dbutils.widgets.text("Vector_Index", "hls_llm_qa_demo.vse.hls_llm_qa_embeddings")
dbutils.widgets.text("Vector_Index", "dbc_adv_anlaytics_dev.yh_agent_resume.resumes_chunked_index_p4")

# Target Catalog Name
#dbutils.widgets.text("Catalog_Name", "hls_llm_qa_demo")
dbutils.widgets.text("Catalog_Name", "dbc_adv_anlaytics_dev")

# Target VSE Schema Name
dbutils.widgets.text("Vse_Schema_Name", "yh_agent_resume")

# Target UC_CATALOG Schema Name
dbutils.widgets.text("UC_CATALOG", "dbc_adv_anlaytics_dev")

# Target UC_SCHEMA Name
dbutils.widgets.text("UC_SCHEMA", "yh_agent_resume")

# Target CHUNKS_DELTA_TABLE Name
dbutils.widgets.text("CHUNKS_DELTA_TABLE", "resumes_chunked_p4")



In [0]:
#get widget values
pdf_path = dbutils.widgets.get("UC_Volume_Path")
source_pdfs = dbutils.widgets.get("Source_Documents")
embeddings_model = dbutils.widgets.get("Embeddings_Model")
vector_search_endpoint_name = dbutils.widgets.get("Vector_Search_Endpoint")
vector_index_name = dbutils.widgets.get("Vector_Index")
UC_table_save_location = dbutils.widgets.get("Persisted_UC_Table_Location")
Persisted_UC_Table_Location = dbutils.widgets.get("Persisted_UC_Table_Location")
Vse_Schema_Name = dbutils.widgets.get("Vse_Schema_Name")
UC_CATALOG = dbutils.widgets.get("UC_CATALOG")
UC_SCHEMA = dbutils.widgets.get("UC_SCHEMA")
CHUNKS_DELTA_TABLE = dbutils.widgets.get("CHUNKS_DELTA_TABLE")


# TEMPORARY - NEED TO ADD STRING LOGIC TO GENERATE:
volume_path = pdf_path

In [0]:
print(f"pdf_path = {pdf_path}")
print(f"source_pdfs = {source_pdfs}")
print(f"embeddings_model = {embeddings_model}")
print(f"vector_search_endpoint_name = {vector_search_endpoint_name}")
print(f"vector_index_name = {vector_index_name}")
print(f"UC_table_save_location = {UC_table_save_location}")
print(f"Persisted_UC_Table_Location = {Persisted_UC_Table_Location}")
print(f"Vse_Schema_Name = {Vse_Schema_Name}")
print(f"volume_path = {volume_path}")
print(f"UC_CATALOG = {UC_CATALOG}")
print(f"UC_SCHEMA = {UC_SCHEMA}")
print(f"CHUNKS_DELTA_TABLE = {CHUNKS_DELTA_TABLE}")

pdf_path = /Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume
source_pdfs = /Workspace/Users/yhu01@blueshieldca.com/adb_rag_resume/Resumes_10.8
embeddings_model = databricks-bge-large-en
vector_search_endpoint_name = yhu01_resume_search
vector_index_name = dbc_adv_anlaytics_dev.yh_agent_resume.resumes_chunked_index_p4
UC_table_save_location = dbc_adv_anlaytics_dev.yh_agent_resume.p4_resumes_chunked
Persisted_UC_Table_Location = dbc_adv_anlaytics_dev.yh_agent_resume.p4_resumes_chunked
Vse_Schema_Name = yh_agent_resume
volume_path = /Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume
UC_CATALOG = dbc_adv_anlaytics_dev
UC_SCHEMA = yh_agent_resume
CHUNKS_DELTA_TABLE = resumes_chunked_p4


Create Unity schema if it does not exist

In [0]:
%sql
-- Create Unity schema if it does not exist in the Unity catalog
-- Use IF NOT EXISTS clause to avoid errors if the schema already exists
CREATE SCHEMA IF NOT EXISTS ${Catalog_Name}.yh_agent_resume

## Data Prep

This data preparation need only happen one time to create data sets that can then be reused in later sections without re-running this part.

- Grab the set of PDFs (ex: Arxiv papers allow curl, PubMed does not)
- We have are providing a set of PDFs from PubMedCentral relating to Cystic Fibrosis (all from [PubMedCentral Open Access](https://www.ncbi.nlm.nih.gov/pmc/tools/openftlist/), all with the CC BY license), but any topic area would work
- If you already have a repository of PDFs then you can skip this step, just organize them all in an accessible Unity Catalog Volume

In [0]:
%sql
-- Create an external volume under the newly created directory
CREATE SCHEMA IF NOT EXISTS ${Catalog_Name}.${Vse_Schema_Name}

In [0]:
source_pdfs

'/Workspace/Users/yhu01@blueshieldca.com/adb_rag_resume/Resumes_10.8'

In [0]:
print('file:'+source_pdfs)

file:/Workspace/Users/yhu01@blueshieldca.com/adb_rag_resume/Resumes_10.8


In [0]:
volume_path

'/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume'

copy the files to the volume

In [0]:
dbutils.fs.ls(f'file:'+source_pdfs)

[FileInfo(path='file:/Workspace/Users/yhu01@blueshieldca.com/adb_rag_resume/Resumes_10.8/Venkatesh Terikuti.pdf', name='Venkatesh Terikuti.pdf', size=228198, modificationTime=1759970427976),
 FileInfo(path='file:/Workspace/Users/yhu01@blueshieldca.com/adb_rag_resume/Resumes_10.8/Vivek Reddy.pdf', name='Vivek Reddy.pdf', size=297823, modificationTime=1759970428018),
 FileInfo(path='file:/Workspace/Users/yhu01@blueshieldca.com/adb_rag_resume/Resumes_10.8/ChunMin Jen.pdf', name='ChunMin Jen.pdf', size=63014, modificationTime=1759970428043),
 FileInfo(path='file:/Workspace/Users/yhu01@blueshieldca.com/adb_rag_resume/Resumes_10.8/Parisa Sahraeian.pdf', name='Parisa Sahraeian.pdf', size=168163, modificationTime=1759970428054),
 FileInfo(path='file:/Workspace/Users/yhu01@blueshieldca.com/adb_rag_resume/Resumes_10.8/Mohammad Hassanpour.pdf', name='Mohammad Hassanpour.pdf', size=65326, modificationTime=1759970428060),
 FileInfo(path='file:/Workspace/Users/yhu01@blueshieldca.com/adb_rag_resume/R

clean up the volume

In [0]:
# Source directory in workspace
source_dir = 'file:'+source_pdfs

# Destination volume path
destination_volume = volume_path

#clean up the volume

destination_dir = volume_path
# List files in the source directory
files = dbutils.fs.ls(destination_dir)

# Remove eachj file in the volume
for file_info in files:
    source_path = file_info.path
    destination_path = f"{destination_volume}/{file_info.name}"
    dbutils.fs.rm(destination_path)

In [0]:
#Move the documents to volume on unity catalog

# List files in the source directory
files = dbutils.fs.ls(source_dir)

# Copy each file to the volume
for file_info in files:
    source_path = file_info.path
    destination_path = f"{destination_volume}/{file_info.name}"
    dbutils.fs.cp(source_path, destination_path)

All of the PDFs should now be accessible in the `Unity Catalog Volume` now; you can run the below command to check if you want.

`dbutils.fs.ls(volume_path)`

In [0]:
dbutils.fs.ls(volume_path)

[FileInfo(path='dbfs:/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/Apoorva Tyagi.pdf', name='Apoorva Tyagi.pdf', size=257953, modificationTime=1759982541000),
 FileInfo(path='dbfs:/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/ChunMin Jen.pdf', name='ChunMin Jen.pdf', size=63014, modificationTime=1759982540000),
 FileInfo(path='dbfs:/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/Jia Yang.pdf', name='Jia Yang.pdf', size=405286, modificationTime=1759982541000),
 FileInfo(path='dbfs:/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/Mohammad Hassanpour.pdf', name='Mohammad Hassanpour.pdf', size=65326, modificationTime=1759982540000),
 FileInfo(path='dbfs:/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/Parisa Sahraeian.pdf', name='Parisa Sahraeian.pdf', size=168163, modificationTime=1759982540000),
 FileInfo(path='dbfs:/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/Venkatesh Terikuti.pdf', nam

## Prepare Document DB

Now it's time to load the texts that have been generated, and create a searchable database of text for use in the `langchain` pipeline. 
These documents are embedded, so that later queries can be embedded too, and matched to relevant text chunks by embedding.

- Use `langchain` to reading directly from PDFs, although LangChain also supports txt, HTML, Word docs, GDrive, PDFs, etc.
- Create a Databricks Vector Search endpoint to have a persistent vector index.
- Use the Foundation Model APIs to generate the embeddings to sync against the vector index.
- Sync the vector index to populate for our rag implementation.

Create the document database:
- Here we are using the `PyPDFDirectoryLoader` loader from LangChain ([docs page](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/pdf.html#using-pypdf)) to form `documents`; `langchain` can also form doc collections directly from PDFs, GDrive files, etc.

In [0]:
pdf_path

'/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume'

In [0]:
# Load the volume of pdf's into a list of text.  

from langchain.docstore.document import Document
from langchain.document_loaders import PyPDFDirectoryLoader

# Load directly from Unity Catalog Volumes
loader_path = volume_path

pdf_loader = PyPDFDirectoryLoader(loader_path)
docs = pdf_loader.load()
len(docs)

2025-10-09 04:02:42.135163: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-09 04:02:42.137654: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-09 04:02:42.256224: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-09 04:02:42.380744: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759982562.503396 1320943 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759982562.55

14

In [0]:
loader_path

'/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume'

In [0]:
type(docs[0])

langchain_core.documents.base.Document

Here we are using a text splitter from LangChain to split our PDFs into manageable chunks. This is for a few reasons, primarily:
- LLMs (currently) have a limited context length. DRBX by default has a context length of 32k tokens. tokens (roughly words) in the prompt.
- When we create embeddings for these documents, an NLP model (sentence transformer) creates a numerical representation (a high-dimensional vector) of that chunk of text that captures the semantic meaning of what is being embedded. If we were to embed large documents, the NLP model would need to capture the meaning of the entire document in one vector; by splitting the document, we can capture the meaning of chunks throughout that document and retrieve only what is most relevant.
- In this case, the embeddings model we use can except a very limited number of tokens. 
- More info on embeddings: [Hugging Face: Getting Started with Embeddings](https://huggingface.co/blog/getting-started-with-embeddings)

In [0]:
# For PDFs we need to split them for embedding:
from langchain.text_splitter import TokenTextSplitter

# this is splitting into chunks based on a fixed number of tokens
# the embeddings model we use below can take a maximum of 128 tokens (and truncates beyond that) so we keep our chunks at that max size
text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=32)
documents = text_splitter.split_documents(docs)

Drop the chunked table


In [0]:
%sql
-- Create Unity schema if it does not exist in the Unity catalog
-- Use IF NOT EXISTS clause to avoid errors if the schema already exists
drop table IF EXISTS dbc_adv_anlaytics_dev.yh_agent_resume.resumes_chunked_p4

Load the pandas data into a spark dataframe and then write to the chunked table

In [0]:
from pyspark.sql import SparkSession
from databricks.vector_search.client import VectorSearchClient
from pyspark.sql.functions import monotonically_increasing_id

# Workspace URL for printing links to the delta table/vector index
workspace_url = SparkSession.getActiveSession().conf.get(
    "spark.databricks.workspaceUrl", None
)

# Vector Search client
vsc = VectorSearchClient(disable_notice=True)

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import Row

# Define the schema explicitly
schema = StructType([
    StructField("text", StringType(), True)
])

# Assuming 'docs' is a list of strings
rows = [Row(text=doc) for doc in documents]

# Create a DataFrame with the defined schema
chunked_docs_df = spark.createDataFrame(rows, schema)

# Rename columns and add an 'id' column for the primary key
documents_with_id = chunked_docs_df.withColumnRenamed("text", "chunked_text") \
                            .withColumn("chunk_id", monotonically_increasing_id())

In [0]:

display(documents_with_id)

chunked_text,chunk_id
"page_content='Apoorva Tyagi Milpitas, CA | (669) 264 9455 | apoorva.tyagi0195@gmail.com| http://linkedin.com/in/apoorva-tyagi19 EDUCATION California State University, East Bay, California, USA ' metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-06-30T16:05:31-07:00', 'author': 'Windows User', 'moddate': '2025-06-30T16:05:31-07:00', 'source': '/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/Apoorva Tyagi.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}",0
"page_content=' EDUCATION California State University, East Bay, California, USA ' metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-06-30T16:05:31-07:00', 'author': 'Windows User', 'moddate': '2025-06-30T16:05:31-07:00', 'source': '/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/Apoorva Tyagi.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}",1
"page_content=' Aug 2022 – May 2024 Master of Science in Data/Business Analytics ' metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-06-30T16:05:31-07:00', 'author': 'Windows User', 'moddate': '2025-06-30T16:05:31-07:00', 'source': '/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/Apoorva Tyagi.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}",2
"page_content=' Birla Institute of Management and Technology (BIMTECH) ' metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-06-30T16:05:31-07:00', 'author': 'Windows User', 'moddate': '2025-06-30T16:05:31-07:00', 'source': '/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/Apoorva Tyagi.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}",3
"page_content=' Birla Institute of Management and Technology (BIMTECH) Aug 2017 – Mar 2019' metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-06-30T16:05:31-07:00', 'author': 'Windows User', 'moddate': '2025-06-30T16:05:31-07:00', 'source': '/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/Apoorva Tyagi.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}",4
"page_content=' Aug 2017 – Mar 2019 Master of Business Administration (MBA) ' metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-06-30T16:05:31-07:00', 'author': 'Windows User', 'moddate': '2025-06-30T16:05:31-07:00', 'source': '/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/Apoorva Tyagi.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}",5
"page_content=' ' metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-06-30T16:05:31-07:00', 'author': 'Windows User', 'moddate': '2025-06-30T16:05:31-07:00', 'source': '/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/Apoorva Tyagi.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}",6
"page_content=' University of Delhi ' metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-06-30T16:05:31-07:00', 'author': 'Windows User', 'moddate': '2025-06-30T16:05:31-07:00', 'source': '/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/Apoorva Tyagi.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}",7
"page_content=' ' metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-06-30T16:05:31-07:00', 'author': 'Windows User', 'moddate': '2025-06-30T16:05:31-07:00', 'source': '/Volumes/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resume_volume/Apoorva Tyagi.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}",8
"page_content=' Jul 2013 – May 2016 Bachelors in Finance ' metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-06-30T16:05:31-07:00', 'author': 'Windows User', 'moddate': '2025-06-30T16:05:31-07:00', 'sourc

Now that we split the documents into more manageable chunks. We will now set up **Databricks Vector Search** with a **Direct Vector Access Index** which will be used with Langchain in our RAG architecture. 
- We first need to create a dataframe with an id column to be used with Vector Search.
- We will then calculate the embeddings using BGE
- Finally we will save this in our Vector Search as an index to be used for RAG.

In [0]:
UC_CATALOG = f'dbc_adv_anlaytics_dev'
UC_SCHEMA = f'yh_agent_resume'

In [0]:


CHUNKS_DELTA_TABLE = f"{UC_CATALOG}.{UC_SCHEMA}.p4_resumes_chunked"
CHUNKS_VECTOR_INDEX = f"{UC_CATALOG}.{UC_SCHEMA}.p4_resumes_chunked_index"

In [0]:
VECTOR_SEARCH_ENDPOINT = vector_search_endpoint_name

In [0]:
CHUNKS_VECTOR_INDEX

'dbc_adv_anlaytics_dev.yh_agent_resume.p4_resumes_chunked_index'

This is setting up the catalog for the chunked data as well as establishes the Vector Search endpoint

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.vectorsearch import EndpointStatusState, EndpointType
from databricks.sdk.service.serving import EndpointCoreConfigInput, EndpointStateReady
from databricks.sdk.errors import ResourceDoesNotExist, NotFound, PermissionDenied
import os
w = WorkspaceClient()

# Create UC Catalog if it does not exist, otherwise, raise an exception
try:
    _ = w.catalogs.get(UC_CATALOG)
    print(f"PASS: UC catalog `{UC_CATALOG}` exists")
except NotFound as e:
    print(f"`{UC_CATALOG}` does not exist, trying to create...")
    try:
        _ = w.catalogs.create(name=UC_CATALOG)
    except PermissionDenied as e:
        print(f"FAIL: `{UC_CATALOG}` does not exist, and no permissions to create.  Please provide an existing UC Catalog.")
        raise ValueError(f"Unity Catalog `{UC_CATALOG}` does not exist.")
        
# Create UC Schema if it does not exist, otherwise, raise an exception
try:
    _ = w.schemas.get(full_name=f"{UC_CATALOG}.{UC_SCHEMA}")
    print(f"PASS: UC schema `{UC_CATALOG}.{UC_SCHEMA}` exists")
except NotFound as e:
    print(f"`{UC_CATALOG}.{UC_SCHEMA}` does not exist, trying to create...")
    try:
        _ = w.schemas.create(name=UC_SCHEMA, catalog_name=UC_CATALOG)
        print(f"PASS: UC schema `{UC_CATALOG}.{UC_SCHEMA}` created")
    except PermissionDenied as e:
        print(f"FAIL: `{UC_CATALOG}.{UC_SCHEMA}` does not exist, and no permissions to create.  Please provide an existing UC Schema.")
        raise ValueError("Unity Catalog Schema `{UC_CATALOG}.{UC_SCHEMA}` does not exist.")

# Create the Vector Search endpoint if it does not exist
vector_search_endpoints = w.vector_search_endpoints.list_endpoints()
if sum([VECTOR_SEARCH_ENDPOINT == ve.name for ve in vector_search_endpoints]) == 0:
    print(f"Please wait, creating Vector Search endpoint `{VECTOR_SEARCH_ENDPOINT}`.  This can take up to 20 minutes...")
    w.vector_search_endpoints.create_endpoint_and_wait(VECTOR_SEARCH_ENDPOINT, endpoint_type=EndpointType.STANDARD)

# Make sure vector search endpoint is online and ready.
w.vector_search_endpoints.wait_get_endpoint_vector_search_endpoint_online(VECTOR_SEARCH_ENDPOINT)

print(f"PASS: Vector Search endpoint `{VECTOR_SEARCH_ENDPOINT}` exists")

PASS: UC catalog `dbc_adv_anlaytics_dev` exists
PASS: UC schema `dbc_adv_anlaytics_dev.yh_agent_resume` exists
PASS: Vector Search endpoint `yhu01_resume_search` exists


In [0]:
CHUNKS_DELTA_TABLE

'dbc_adv_anlaytics_dev.yh_agent_resume.p4_resumes_chunked'

In [0]:
# Now you can proceed with saving chunked_docs_df to a Delta table for further processing
documents_with_id.write.format("delta").mode("overwrite").saveAsTable(CHUNKS_DELTA_TABLE)

# Enable change-data capture for the Delta table
spark.sql(f"ALTER TABLE {CHUNKS_DELTA_TABLE} SET TBLPROPERTIES (delta.enableChangeDataFeed = true)")

print(f"View Delta Table at: https://{workspace_url}/explore/data/{UC_CATALOG}/{UC_SCHEMA}/{CHUNKS_DELTA_TABLE.split('.')[-1]}")

View Delta Table at: https://adb-640321604414221.1.azuredatabricks.net/explore/data/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resumes_chunked


THIS is the end that has written to the chunk table now we need to run the code to create the index

1/ Create the Vector Search Index
First, we copy the sample data to a Delta Table and sync to a Vector Search index. Here, we use the gte-large-en-v1.5 embedding model hosted on Databricks Foundational Model APIs.

In [0]:

index_name=vector_index_name
index_name

# Attempt to delete the existing index if it exists
try:
    vsc.delete_index(index_name=index_name)
    print(f"Existing index '{index_name}' deleted successfully.")
except Exception as e:
    print(f"No existing index named '{index_name}' was found or an error occurred: {e}")


No existing index named 'dbc_adv_anlaytics_dev.yh_agent_resume.resumes_chunked_index_p4' was found or an error occurred: Response content b'{"error_code":"RESOURCE_DOES_NOT_EXIST","message":"Unity Catalog entity dbc_adv_anlaytics_dev.yh_agent_resume.resumes_chunked_index_p4 does not exist.","details":[{"@type":"type.googleapis.com/google.rpc.RequestInfo","request_id":"e932d44d-7f9d-4f50-bacf-b159331a7bbc","serving_data":""}]}', status_code 404


In [0]:
from databricks.vector_search.client import VectorSearchClient

# Embed and sync chunks to a vector index
print(
    f"Embedding docs & creating Vector Search Index, this will take ~5 - 10 minutes.\nView Index Status at: https://{workspace_url}/explore/data/{UC_CATALOG}/{UC_SCHEMA}/{CHUNKS_VECTOR_INDEX.split('.')[-1]}"
)

# Initialize the Vector Search client
vsc = VectorSearchClient(disable_notice=True)

# Define the index name and other parameters
index_name = CHUNKS_VECTOR_INDEX
source_table_name = CHUNKS_DELTA_TABLE
primary_key_column = "chunk_id"  # Adjust based on your table's primary key
embedding_column = "chunked_text"  # Adjust based on your embedding column

# Check if the source table exists
if not spark.catalog.tableExists(source_table_name):
    raise Exception(f"Table '{source_table_name}' does not exist.")

# Proceed to create the Vector Search index without invoking serverless compute
# This example creates a Delta Sync Index with embeddings computed by Databricks
try:
    index = vsc.create_delta_sync_index_and_wait(
        endpoint_name=VECTOR_SEARCH_ENDPOINT,
        index_name=index_name,
        primary_key=primary_key_column,
        source_table_name=source_table_name,
        pipeline_type="TRIGGERED",  # Using TRIGGERED to avoid continuous compute costs
        embedding_source_column=embedding_column,
        embedding_model_endpoint_name="databricks-bge-large-en",  # Specify your model endpoint
    )
    print(f"Vector Search index '{index_name}' created successfully.")
except Exception as e:
    print(f"Error creating Vector Search index '{index_name}': {e}")

Embedding docs & creating Vector Search Index, this will take ~5 - 10 minutes.
View Index Status at: https://adb-640321604414221.1.azuredatabricks.net/explore/data/dbc_adv_anlaytics_dev/yh_agent_resume/p4_resumes_chunked_index
Error creating Vector Search index 'dbc_adv_anlaytics_dev.yh_agent_resume.p4_resumes_chunked_index': Response content b'{"error_code":"RESOURCE_ALREADY_EXISTS","message":"UC entity dbc_adv_anlaytics_dev.yh_agent_resume.p4_resumes_chunked_index already exists. Please use a unique name in UC for the index name.","details":[{"@type":"type.googleapis.com/google.rpc.RequestInfo","request_id":"67d787d7-c2da-4afc-b1f4-0dc954b104eb","serving_data":""}]}', status_code 400


2/ Deploy to the review application

Now that our Vector Search index is ready, let's prepare the RAG chain and deploy it to the review application backed by a scalable-production ready REST API on Model serving.

2.1/ Configuring our Chain parameters
Databricks makes it easy to parameterize your chain with MLflow Model Configurations. Later, you can tune application quality by adjusting these parameters, such as the system prompt or retrieval settings. Most applications will include many more parameters, but for this demo, we'll keep the configuration to a minimum.

In [0]:
chain_config = {
    "llm_model_serving_endpoint_name": "poc-openai-completions-endpoint",  # the foundation model we want to use
    "vector_search_endpoint_name": VECTOR_SEARCH_ENDPOINT,  # Endoint for vector search
    "vector_search_index": f"{CHUNKS_VECTOR_INDEX}",
    "llm_prompt_template": """You are a hiring manager for Data Scientist, Consultant of Blue Shield of California, which 
-Requires a bachelor’s degree in mathematics, statistics, computer science, or an equivalent quantitative scientific discipline
-Requires at least 3 years of professional Data Science or ML experience; or a Ph.D. degree in operations research, applied statistics, data mining, machine learning, or other quantitative discipline
-Must be able to demonstrate real-world experience to translate business problems into an ML problem and be able to communicate AI recommendations in a business context to a general non-technical audience
-Proficiency in scalable data transformation techniques using SQL, SAS, Spark, or equivalent
-Proficiency in open-source languages such as Python, R, and Julia
-Hands-on experience with cloud environments such as Azure and Google Cloud and nice to have experience in DataBricks
-Understanding of statistical methods and advanced modeling techniques (e.g., SVM, K-Means, Random Forest, Boosting, Bayesian inference, natural language processing)
-Experience with machine learning and deep learning packages (scikit-learn, XGBoost, Tensorflow, or PyTorch)
-Knowledge of evaluating solutions for fairness, bias, accuracy, drift, validity, fit, robustness, and explainability
-MLOps experience including good design documentation, unit testing, integration testing, and version control (git)
-Experience in experimentation design and A/B testing
-Knowledge and experience in Generative AI techniques and applications, including natural language generation, image synthesis, and automated content creation
-Experience with Generative AI frameworks and tools such as GPT, GANs, and VAEs
-Ability to develop and deploy generative models for various applications
-Understanding of ethical considerations and best practices in the development and deployment of Generative AI solutions
-Ability to partner, collaborate with, and lead relevant stakeholders across diverse functions and experience levels 


Job Description
The Advanced Analytics team works in partnership across the entire Blue Shield of CA enterprise to accelerate business outcomes through the application of AI/machine learning, statistical methodologies, or unstructured data analysis techniques to uncover insights, predict behaviors, and ultimately drive automation to create “intelligence at scale”. The Data Scientist, Consultant will report to the Director, Advanced Analytics. In this role you will solve problems which range from but are not limited to, text analytics of customer feedback, conversations, and clinical notes, predicting clinical disease progression, understanding the impact of population health programs, clustering member behaviors, creating propensity models, and geospatial analysis of populations to uncover social determinants of health.

Responsibilities
Collaborate with product owners and business stakeholders to identify opportunities for process optimization and decision-making improvements
Perform data exploration and visualization using Python on DataBricks, Tableau, and Python Notebooks to understand the signal-to-noise ratio in datasets
Partner with the data engineering team for rapid prototyping of training data sets using SQL, Apache Spark, and other tools
Conduct feature engineering using appropriate techniques for the given data and business problem
Develop robust model validation procedures and generate performance metrics for evaluation and monitoring
Act as the subject matter expert in applied ML for claims, premiums, member risk scores, and other business contexts
Drive the lifecycle of machine learning projects from ideation to deployment, ensuring timely delivery and maintaining documentation
Perform literature reviews to inform statistical modeling approaches and best practices
Translate business requirements into technical specifications and design scalable pipelines with data engineers
Assist in transitioning from on-prem infrastructure to the cloud
Convert text-based data into feature data sets for predictive analytics
Develop materials to explain project findings
Lead and contribute to Generative AI projects, including model development and deployment for applications like natural language generation and image synthesis
Stay updated with advancements in Generative AI and incorporate relevant techniques into projects
Collaborate with cross-functional teams to integrate Generative AI solutions into products and services
Conduct experiments and research to explore new applications of Generative AI in healthcare
Provide mentorship on Generative AI techniques and best practices
Perform other projects or duties as assigned

Use the following resumes answer the question. \n\nContext: {context}""", # LLM Prompt template
}

# Here, we define an input example in the schema required by Agent Framework
input_example = {"messages": [ {"role": "user", "content": "Who would be a good data scientist?"}]}

2.1/ Log the application & view trace
We first register the chain as an MLflow model and inspect the MLflow Trace to understand what is happening inside the chain.

MLflow trace

In [0]:
QUICK_START_REPO_SAVE_FOLDER = "genai-resume"

import os
os.getcwd()
lc_model=os.path.join(
            os.getcwd(),
            f"rag_langchain_runner",
        ),  # Chain code file from the quick start repo

In [0]:
lc_model

('/Workspace/Users/yhu01@blueshieldca.com/adb_rag_resume/rag_langchain_runner',)

In [0]:
import mlflow

# Log the model to MLflow
with mlflow.start_run(run_name="databricks-docs-bot"):
    logged_chain_info = mlflow.langchain.log_model(
        lc_model=lc_model[0],  # Chain code file from the quick start repo
        model_config=chain_config,  # Chain configuration set above
        artifact_path="chain",  # Required by MLflow
        input_example=input_example,  # Save the chain's input schema.  MLflow will execute the chain before logging & capture it's output schema.
    )

# Test the chain locally to see the MLflow Trace
chain = mlflow.langchain.load_model(logged_chain_info.model_uri)
chain.invoke(input_example)

2025/10/09 04:03:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
🔗 View Logged Model at: https://adb-640321604414221.1.azuredatabricks.net/ml/experiments/eaeb0ef61f564c63a0c453968c21069a/models/m-ae7082f6457a4fa4896e95aab0f89387?o=640321604414221
2025/10/09 04:03:34 WARNING mlflow.models.utils: The model file uses 'dbutils' commands which are not supported. To ensure your code functions correctly, make sure that it does not rely on these dbutils commands for correctness.
/tmp/tmpdfqkp7t1/model.py:64: LangChainDeprecationWarning: The class `DatabricksVectorSearch` was deprecated in LangChain 0.3.3 and will be removed in 1.0. An updated version of the class exists in the :class:`~databricks-langchain package and should be used instead. To use it run `pip install -U :class:`~databricks-langchain` and import as `from :class:`~databricks_langchain import DatabricksVectorSearch``.
  vector_search_as_retriever = DatabricksVectorSearch(
/tmp/tmpdfqkp7t

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


2025/10/09 04:03:36 INFO mlflow: Attempting to auto-detect Databricks resource dependencies for the current langchain model. Dependency auto-detection is best-effort and may not capture all dependencies of your langchain model, resulting in authorization errors when serving or querying your model. We recommend that you explicitly pass `resources` to mlflow.langchain.log_model() to ensure authorization to dependent resources succeeds when the model is deployed.
2025/10/09 04:04:06 WARNING mlflow.utils.requirements_utils: Failed to run predict on input_example, dependencies introduced in predict are not captured.
MlflowException('1 tasks failed. Errors: {0: \'error: Exception(\\\'Response content b\\\\\\\'{"error_code":"NOT_FOUND","message":"Vector search endpoint f78b32e3-9a0f-4915-b045-21ad2edd3979 not found.","details":[{"@type":"type.googleapis.com/google.rpc.RequestInfo","request_id":"77c176dd-707e-4397-84ff-a196910619dc","serving_data":""}]}\\\\\\\', status_code 404\\\') Traceback 

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


/local_disk0/repl_tmp_data/ReplId-199ab-e2bbb-c/tmpupa_m3jc/model.py:79: FutureWarning: set_retriever_schema is deprecated and will be removed in a future version. Please migrate to use VectorSearchRetrieverTool in the 'databricks-ai-bridge' package, or match the default schema so your retriever spans can be detected without requiring explicit configuration. See https://mlflow.org/docs/latest/genai/data-model/traces/#retriever-spans for more information.
  mlflow.models.set_retriever_schema(


[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


Trace(trace_id=tr-8c9f1677c2d779aca36881ce13df1a34)

---------------------------------------------------------------------------
HTTPError                                 Traceback (most recent call last)
File /local_disk0/.ephemeral_nfs/envs/pythonEnv-88c91d3a-838e-430e-aea5-d5232d32cf7a/lib/python3.12/site-packages/databricks/vector_search/utils.py:159, in RequestUtils.issue_request(url, method, token, params, json, verify, auth, data, headers)
    158 try:
--> 159     response.raise_for_status()
    160 except Exception as e:

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-88c91d3a-838e-430e-aea5-d5232d32cf7a/lib/python3.12/site-packages/requests/models.py:1026, in Response.raise_for_status(self)
   1025 if http_error_msg:
-> 1026     raise HTTPError(http_error_msg, response=self)

HTTPError: 404 Client Error: Not Found for url: https://centralus-c2.azuredatabricks.net/api/2.0/vector-search/indexes/dbc_adv_anlaytics_dev.yh_agent_resume.p4_resumes_chunked_index/query

During handling of the above exception, another exception occurred:

In [0]:
chain_config = {
    "llm_model_serving_endpoint_name": "poc-openai-completions-endpoint",  # the foundation model we want to use
    "vector_search_endpoint_name": VECTOR_SEARCH_ENDPOINT,  # Endoint for vector search
    "vector_search_index": f"{CHUNKS_VECTOR_INDEX}",
    "llm_prompt_template": """you are a manager hiring a Data Scientist. Use the following pieces of retrieved context to answer the question. Some pieces of context may be irrelevant, in which case you should not use them to form the answer.\n\nContext: {context}""", # LLM Prompt template
}

# Here, we define an input example in the schema required by Agent Framework
input_example = {"messages": [ {"role": "user", "content": "who is the best candidate for data scientist role for blue shield of ca?"}]}

py4j.Py4JException: Error while obtaining a new communication channel
	at py4j.CallbackClient.getConnectionLock(CallbackClient.java:257)
	at py4j.CallbackClient.sendCommand(CallbackClient.java:377)
	at py4j.CallbackClient.sendCommand(CallbackClient.java:356)
	at py4j.reflection.PythonProxyHandler.invoke(PythonProxyHandler.java:106)
	at jdk.proxy23/jdk.proxy23.$Proxy217.updateWsfsWorkingDir(Unknown Source)
	at com.databricks.backend.daemon.driver.PythonDriverLocalBase.$anonfun$setWsfsWorkingDir$3(PythonDriverLocalBase.scala:444)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.scala:18)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:687)
	at scala.concurrent.impl.Promise$Transformation.run(Promise.scala:467)
	at java.base/java.util.concurrent.ForkJoinTask$RunnableExecuteAction.exec(ForkJoinTask.java:1395)
	at java.base/java.util.concurrent.ForkJoinTask.doExec(ForkJoinTask.java:373)
	at java.base/java.util.concurrent.ForkJoinPool$WorkQueue.topLevelExec(ForkJo

In [0]:
# Test the chain locally to see the MLflow Trace
chain = mlflow.langchain.load_model(logged_chain_info.model_uri)
chain.invoke(input_example)

2.1/ Deploy the application
Now, we:

Register the application in Unity Catalog
Use Agent Framework to deploy to the Quality Lab review application
Along side the review ap, a scalable, production-ready Model Serving endpoint is also deployed.

In [0]:
import uuid

# Use the current user name to create any necesary resources
w = WorkspaceClient()
user_name = w.current_user.me().user_name.split("@")[0].replace(".", "")

# UC Catalog & Schema where outputs tables/indexs are saved
# If this catalog/schema does not exist, you need create catalog/schema permissions.
UC_CATALOG = f'dbc_adv_anlaytics_dev'
UC_SCHEMA = f'yh_agent_resume'

# UC Model name where the POC chain is logged
UC_MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.chat_bot_updated_{uuid.uuid4().hex[:8]}"

# Vector Search endpoint where index is loaded
# If this does not exist, it will be created
VECTOR_SEARCH_ENDPOINT = f'{user_name}_resume_search'

In [0]:
UC_MODEL_NAME

In [0]:
from databricks import agents
import time
from databricks.sdk.service.serving import EndpointStateReady, EndpointStateConfigUpdate

# Use Unity Catalog to log the chain
import mlflow

# Register the chain to UC
uc_registered_model_info = mlflow.register_model(model_uri=logged_chain_info.model_uri, name=UC_MODEL_NAME)

# Deploy to enable the Review APP and create an API endpoint
deployment_info = agents.deploy(model_name=UC_MODEL_NAME, model_version=uc_registered_model_info.version)

# Wait for the Review App to be ready
print("\nWaiting for endpoint to deploy.  This can take 10 - 20 minutes.", end="")
while w.serving_endpoints.get(deployment_info.endpoint_name).state.ready == EndpointStateReady.NOT_READY or w.serving_endpoints.get(deployment_info.endpoint_name).state.config_update == EndpointStateConfigUpdate.IN_PROGRESS:
    print(".", end="")
    time.sleep(30)